In [1]:
import connection as cn

In [2]:
con = cn.get_session()

In [3]:
con

In [7]:
from sqlalchemy import text

sql = text("SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname = 'analytics';")
result = con.execute(sql)

tabelas = [row[0] for row in result]
print(tabelas)

['dim_date', 'dim_seller', 'dim_product', 'dim_customer', 'fact_sales', 'df_orders', 'df_items']


In [5]:
import pandas as pd

df_orders = pd.read_csv("../data/olist_orders_dataset.csv")
df_customers = pd.read_csv("../data/olist_customers_dataset.csv")
df_items = pd.read_csv("../data/olist_order_items_dataset.csv")
df_products = pd.read_csv("../data/olist_products_dataset.csv")
df_sellers = pd.read_csv("../data/olist_sellers_dataset.csv")


In [6]:
import pandas as pd
from sqlalchemy import text
from connection import get_engine
import os
from pathlib import Path

# Diretórios
NOTEBOOK_DIR = Path(os.getcwd())
DATA_DIR = NOTEBOOK_DIR.parent / "data"
ORDERS_PATH = DATA_DIR / "olist_orders_dataset.csv"

def load_dim_date():
    engine = get_engine()
    print("Conectado ao banco!")

    # 1. Ler CSV
    print(f"Lendo: {ORDERS_PATH}")
    df_orders = pd.read_csv(ORDERS_PATH)

    # padronizar colunas
    df_orders.columns = [c.lower().strip().replace(" ", "_") for c in df_orders.columns]

    # 2. Criar tabela temporária no schema analytics
    df_orders.to_sql(
        "df_orders",
        engine,
        schema="analytics",
        if_exists="replace",
        index=False
    )
    print("Tabela temporária analytics.df_orders criada!")

    # 3. Inserir dados na dim_date
    insert_sql = text("""
        INSERT INTO analytics.dim_date (
            date_sk, full_date, year, month, day,
            day_of_week, month_name, day_name
        )
        SELECT
            ROW_NUMBER() OVER (ORDER BY full_date_clean),
            full_date_clean,
            EXTRACT(YEAR FROM full_date_clean),
            EXTRACT(MONTH FROM full_date_clean),
            EXTRACT(DAY FROM full_date_clean),
            EXTRACT(DOW FROM full_date_clean) + 1,
            TO_CHAR(full_date_clean, 'Month'),
            TO_CHAR(full_date_clean, 'Day')
        FROM (
            SELECT DISTINCT DATE_TRUNC('day', order_purchase_timestamp::timestamp)
            AS full_date_clean
            FROM analytics.df_orders
        ) AS unique_dates;
    """)

    with engine.begin() as conn:
        conn.execute(insert_sql)

    print("dim_date populada com sucesso!")

    # 4. Remover tabela temporária
    with engine.begin() as conn:
        conn.execute(text("DROP TABLE IF EXISTS analytics.df_orders;"))

    print("Tabela temporária removida!")

if __name__ == "__main__":
    load_dim_date()


Conectado ao banco!
Lendo: C:\Users\mmsantos\Downloads\db p2\P2-BDA-\projeto-dw-grupo-A\data\olist_orders_dataset.csv
Tabela temporária analytics.df_orders criada!


IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "dim_date_pkey"
DETAIL:  Key (date_sk)=(1) already exists.

[SQL: 
        INSERT INTO analytics.dim_date (
            date_sk, full_date, year, month, day,
            day_of_week, month_name, day_name
        )
        SELECT
            ROW_NUMBER() OVER (ORDER BY full_date_clean),
            full_date_clean,
            EXTRACT(YEAR FROM full_date_clean),
            EXTRACT(MONTH FROM full_date_clean),
            EXTRACT(DAY FROM full_date_clean),
            EXTRACT(DOW FROM full_date_clean) + 1,
            TO_CHAR(full_date_clean, 'Month'),
            TO_CHAR(full_date_clean, 'Day')
        FROM (
            SELECT DISTINCT DATE_TRUNC('day', order_purchase_timestamp::timestamp)
            AS full_date_clean
            FROM analytics.df_orders
        ) AS unique_dates;
    ]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [ ]:
import pandas as pd
from sqlalchemy import text
from connection import get_engine
import os
from pathlib import Path

# Diretórios
NOTEBOOK_DIR = Path(os.getcwd())
DATA_DIR = NOTEBOOK_DIR.parent / "data"
CUSTOMERS_PATH = DATA_DIR / "olist_customers_dataset.csv"

def load_dim_customer():
    engine = get_engine()
    print("Conectado ao banco!")

    # 1. Ler CSV
    print(f"Lendo: {CUSTOMERS_PATH}")
    df_customers = pd.read_csv(CUSTOMERS_PATH)

    # padronizar colunas
    df_customers.columns = [c.lower().strip().replace(" ", "_") for c in df_customers.columns]

    # 2. Criar tabela temporária no schema analytics
    df_customers.to_sql(
        "df_customers",
        engine,
        schema="analytics",
        if_exists="replace",
        index=False
    )
    print("Tabela temporária analytics.df_customers criada!")

    # 3. Inserir dados na dim_customer
    insert_sql = text("""
        INSERT INTO analytics.dim_customer (
            customer_id,
            customer_unique_id,
            customer_zip_code_prefix,
            customer_city,
            customer_state
        )
        SELECT
            customer_id,
            customer_unique_id,
            customer_zip_code_prefix,
            customer_city,
            customer_state
        FROM analytics.df_customers;
    """)

    with engine.begin() as conn:
        conn.execute(insert_sql)

    print("dim_customer populada com sucesso!")

    # 4. Remover tabela temporária
    with engine.begin() as conn:
        conn.execute(text("DROP TABLE IF EXISTS analytics.df_customers;"))

    print("Tabela temporária removida!")

if __name__ == "__main__":
    load_dim_customer()


In [ ]:
import pandas as pd
from sqlalchemy import text
from connection import get_engine
import os
from pathlib import Path

# Diretórios
NOTEBOOK_DIR = Path(os.getcwd())
DATA_DIR = NOTEBOOK_DIR.parent / "data"
SELLERS_PATH = DATA_DIR / "olist_sellers_dataset.csv"

def load_dim_seller():
    engine = get_engine()
    print("Conectado ao banco!")

    # 1. Ler CSV
    print(f"Lendo: {SELLERS_PATH}")
    df_sellers = pd.read_csv(SELLERS_PATH)

    # padronizar nomes de colunas
    df_sellers.columns = [c.lower().strip().replace(" ", "_") for c in df_sellers.columns]

    # 2. Criar tabela temporária no schema analytics
    df_sellers.to_sql(
        "df_sellers",
        engine,
        schema="analytics",
        if_exists="replace",
        index=False
    )
    print("Tabela temporária analytics.df_sellers criada!")

    # 3. Inserir dados na dim_seller
    insert_sql = text("""
        INSERT INTO analytics.dim_seller (
            seller_id,
            seller_zip_code_prefix,
            seller_city,
            seller_state
        )
        SELECT
            seller_id,
            seller_zip_code_prefix,
            seller_city,
            seller_state
        FROM analytics.df_sellers;
    """)

    with engine.begin() as conn:
        conn.execute(insert_sql)

    print("dim_seller populada com sucesso!")

    # 4. Remover tabela temporária
    with engine.begin() as conn:
        conn.execute(text("DROP TABLE IF EXISTS analytics.df_sellers;"))

    print("Tabela temporária removida!")

if __name__ == "__main__":
    load_dim_seller()


In [ ]:
import pandas as pd
from sqlalchemy import text
from connection import get_engine
import os
from pathlib import Path

# Diretórios
NOTEBOOK_DIR = Path(os.getcwd())
DATA_DIR = NOTEBOOK_DIR.parent / "data"
PRODUCTS_PATH = DATA_DIR / "olist_products_dataset.csv"

def load_dim_product():
    engine = get_engine()
    print("Conectado ao banco!")

    # 1. Ler CSV
    print(f"Lendo: {PRODUCTS_PATH}")
    df_products = pd.read_csv(PRODUCTS_PATH)

    # padronizar nomes das colunas
    df_products.columns = [c.lower().strip().replace(" ", "_") for c in df_products.columns]

    # 2. Criar tabela temporária df_products no schema analytics
    df_products.to_sql(
        "df_products",
        engine,
        schema="analytics",
        if_exists="replace",
        index=False
    )
    print("Tabela temporária analytics.df_products criada!")

    # 3. Inserir dados na dim_product
    insert_sql = text("""
        INSERT INTO analytics.dim_product (
            product_id,
            product_category_name,
            product_name_lenght,
            product_description_lenght,
            product_photos_qty,
            product_weight_g,
            product_length_cm,
            product_height_cm,
            product_width_cm
        )
        SELECT
            product_id,
            product_category_name,
            product_name_lenght,
            product_description_lenght,
            product_photos_qty,
            product_weight_g,
            product_length_cm,
            product_height_cm,
            product_width_cm
        FROM analytics.df_products;
    """)

    with engine.begin() as conn:
        conn.execute(insert_sql)

    print("dim_product populada com sucesso!")

    # 4. Remover tabela temporária
    with engine.begin() as conn:
        conn.execute(text("DROP TABLE IF EXISTS analytics.df_products;"))

    print("Tabela temporária removida!")

if __name__ == "__main__":
    load_dim_product()


In [9]:
import pandas as pd
from sqlalchemy import text
from connection import get_engine
import os
from pathlib import Path

# Diretório do notebook
NOTEBOOK_DIR = Path(os.getcwd())
DATA_DIR = NOTEBOOK_DIR.parent / "data"

# Arquivos
ORDERS_PATH = DATA_DIR / "olist_orders_dataset.csv"
ITEMS_PATH = DATA_DIR / "olist_order_items_dataset.csv"


def load_fact_sales():
    engine = get_engine()
    print("🔌 Conectado ao banco!")

    # =====================================================
    # 1. Ler os CSVs
    # =====================================================
    print("📄 Carregando arquivos...")
    df_orders = pd.read_csv(ORDERS_PATH)
    df_items = pd.read_csv(ITEMS_PATH)

    # =====================================================
    # 2. Padronizar nomes
    # =====================================================
    df_orders.columns = [c.lower().strip().replace(" ", "_") for c in df_orders.columns]
    df_items.columns = [c.lower().strip().replace(" ", "_") for c in df_items.columns]

    # =====================================================
    # 3. Criar Tabelas Temporárias
    # =====================================================
    print("📌 Criando tabelas temporárias...")

    df_orders.to_sql(
        "temp_orders",
        engine,
        schema="analytics",
        if_exists="replace",
        index=False
    )

    df_items.to_sql(
        "temp_items",
        engine,
        schema="analytics",
        if_exists="replace",
        index=False
    )

    print("✅ Tabelas temporárias criadas!")

    # =====================================================
    # 4. Inserção com CASTs seguros
    # =====================================================

    insert_sql = text("""
        INSERT INTO analytics.fact_sales (
            order_id,
            customer_id,
            order_purchase_timestamp,
            order_item,
            product_id,
            seller_id,
            shipping_limit_date,
            price,
            freight_value
        )
        SELECT
            o.order_id,
            o.customer_id,
            o.order_purchase_timestamp::timestamp,
            i.order_item_id::integer,
            i.product_id,
            i.seller_id,
            i.shipping_limit_date::timestamp,
            i.price::numeric,
            i.freight_value::numeric
        FROM analytics.temp_orders o
        JOIN analytics.temp_items i
            ON o.order_id = i.order_id;
    """)

    print("📥 Inserindo dados na fact_sales...")

    with engine.begin() as conn:
        conn.execute(insert_sql)

    print("🎉 Dados inseridos com sucesso na fact_sales!")

    # =====================================================
    # 5. Remover tabelas temporárias
    # =====================================================
    print("🧹 Limpando tabelas temporárias...")

    with engine.begin() as conn:
        conn.execute(text("DROP TABLE IF EXISTS analytics.temp_orders;"))
        conn.execute(text("DROP TABLE IF EXISTS analytics.temp_items;"))

    print("🚀 Processo finalizado!")


if __name__ == "__main__":
    load_fact_sales()


🔌 Conectado ao banco!
📄 Carregando arquivos...
📌 Criando tabelas temporárias...
✅ Tabelas temporárias criadas!
📥 Inserindo dados na fact_sales...
🎉 Dados inseridos com sucesso na fact_sales!
🧹 Limpando tabelas temporárias...
🚀 Processo finalizado!


In [ ]:
from sqlalchemy import text

for t in ["df_orders", "df_items"]:
    r = con.execute(text(f"""
    SELECT COUNT(*) FROM analytics.{t};
    """)).fetchone()
    print(t, r[0])


In [ ]:
r = con.execute(text("""
SELECT COUNT(*)
FROM analytics.df_orders o
JOIN analytics.df_items i
    ON o.order_id = i.order_id;
""")).fetchone()

print("Linhas retornadas pelo JOIN:", r[0])


In [8]:
import pandas as pd
from sqlalchemy import text
from connection import get_engine
import os
from pathlib import Path

# Diretório atual do notebook
NOTEBOOK_DIR = Path(os.getcwd())
DATA_DIR = NOTEBOOK_DIR.parent / "data"

ORDERS_PATH = DATA_DIR / "olist_orders_dataset.csv"
ITEMS_PATH = DATA_DIR / "olist_order_items_dataset.csv"

# ============================================================
# 1) Carregar CSVs e mostrar informações importantes
# ============================================================

print("📄 Carregando CSVs...")
df_orders = pd.read_csv(ORDERS_PATH)
df_items = pd.read_csv(ITEMS_PATH)

print("\n🧵 Colunas dos arquivos:")
print("df_orders:", df_orders.columns.tolist())
print("df_items:", df_items.columns.tolist())

# padronizar nomes
df_orders.columns = [c.lower().strip().replace(" ", "_") for c in df_orders.columns]
df_items.columns = [c.lower().strip().replace(" ", "_") for c in df_items.columns]

print("\n🔠 Colunas após padronização:")
print("df_orders:", df_orders.columns.tolist())
print("df_items:", df_items.columns.tolist())

# ============================================================
# 2) Diagnóstico dos tipos e exemplos de valores
# ============================================================

print("\n🔍 Tipos das colunas order_id:")
print("df_orders.order_id:", df_orders["order_id"].dtype)
print("df_items.order_id:", df_items["order_id"].dtype)

print("\n🔍 Exemplos de order_id:")
print("df_orders:", df_orders["order_id"].dropna().head())
print("df_items:", df_items["order_id"].dropna().head())

# ============================================================
# 3) Ver quantos order_id existem em cada tabela
# ============================================================

print("\n📊 Quantidade de IDs:")
print("IDs únicos em df_orders:", df_orders["order_id"].nunique())
print("IDs únicos em df_items:", df_items["order_id"].nunique())

# ============================================================
# 4) Ver quantos order_id realmente batem entre os arquivos
# ============================================================

orders_set = set(df_orders["order_id"].astype(str))
items_set = set(df_items["order_id"].astype(str))

intersection = orders_set.intersection(items_set)

print("\n🧩 Quantidade de order_id que existem nos DOIS CSVs:", len(intersection))

if len(intersection) > 0:
    print("\n🧩 10 valores que batem:")
    print(list(intersection)[:10])

# ============================================================
# 5) Diagnóstico direto no banco (se tabelas temporárias já existirem)
# ============================================================

engine = get_engine()
print("\n🔌 Testando direto no banco...")

with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT COUNT(*) 
        FROM analytics.df_orders o
        JOIN analytics.df_items i
            ON o.order_id = i.order_id;
    """))
    print("\n📥 Resultado do JOIN no banco:", result.scalar())


📄 Carregando CSVs...

🧵 Colunas dos arquivos:
df_orders: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
df_items: ['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']

🔠 Colunas após padronização:
df_orders: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
df_items: ['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']

🔍 Tipos das colunas order_id:
df_orders.order_id: object
df_items.order_id: object

🔍 Exemplos de order_id:
df_orders: 0    e481f51cbdc54678b7cc49136f2d6af7
1    53cdb2fc8bc7dce0b6741e2150273451
2    47770eb9100c2d0c44946d9cf07ec65d
3    949d5b44dbf5de918fe9c16f97b45f8a
4    ad21c59c084